In [2]:
import numpy as np
import pickle
import strawberryfields as sf
from sklearn import svm
from strawberryfields import ops
from strawberryfields.decompositions import takagi
from more_itertools import distinct_permutations
from collections import Counter
import os
from functions import *
sf.store_account('eUAjS1VasR1U1gW1pMSg5Bus3bEXWwA1jLge4QG3')
sf.ping()
os.environ['KMP_DUPLICATE_LIB_OK']='True'

You have successfully authenticated to the platform!
You have successfully authenticated to the platform!


# Creating weighted graphs that fit the hardware

Not all weighted graphs can be encoded in the X8 hardware. We describe below a procedure to generate several kinds of weighted bipartite graphs that fit the hardware.

Let's suppose that $B=UDU^{T}$ is a real symetric matrix with eigenvalues $\{\lambda_j\}$.


The eigenvalues of $A = \begin{pmatrix}
0 & B \\
B & 0
\end{pmatrix}$ are $\{\lambda_j, -\lambda_j\}$, because $ Q^T A Q =\begin{pmatrix}
D & 0 \\
0 & -D
\end{pmatrix}$, with $Q=\frac{1}{\sqrt{2}}\begin{pmatrix}
U & -U \\
U & U
\end{pmatrix}$. 

The singular values of a symmetric matrix are the absolute values of its eigenvalues (see corrollary C.5.2 of http://theanalysisofdata.com/probability/C_5.html#:~:text=If%20A%20is%20a%20symmetric,are%20square%20non%2Dsingular%20matrices.).

The BipartiteGraphEmbed function compiled on the X8 hardware can compile matrices for which one has only one non-zero singular value (eventually repeated multiple times).

From the construction above, that restricts the set $S=\{\lambda_j\}$ to be $S=\{0,d\}$. One can still choose the multiplicity of the $d$ eigenvalue, and adapt the mean number of photons accordingly so that all Sgates have squeezing values $r=1$. 

Different "kinds" of graphs (with different labels) correspond to different number of non-zero singular values. 

In [2]:
num_graphs = 30
graphs = [None]*3*num_graphs
labels = [None]*3*num_graphs
#Generate a set of common random unitaries for transforming the off-diagonal 4x4 block
unitaries = generate_unitaries(num_graphs)

#Generate 8x8 matrices that fit the hardware based on the unitaries
for i in range(1,4): #Going through non-trivial singular values 
    for j in range(num_graphs):
        labels[j+num_graphs*(i-1)] = i
        graphs[j+num_graphs*(i-1)] = generate_matrices(i, unitaries[j])

### Remark
Let's note that the program also compiles successfully if one provides a more general matrix $B=UDV^T$ with $U$ and $V$ distinct unitaries. However, when sent to the hardware, the programm cannot be executed because of the absence of symmetry between operations applied on the top and on the bottom of the chip.

# Generating kernel (fingerprint) for each graph

In [3]:
#Fingerprinting function

def fingerprint(samples, num_photons):
    vector_basis = partition(num_photons)
    empty_vector = orbitals(vector_basis)
    clean_sample = truncate_samples(samples)
    prob_dic = estimate_probs(clean_sample)
    full_vector = vector_coordinates(prob_dic,empty_vector)
    #print('full vector is:')
    return full_vector

In [4]:
#Created program for X8 chip
simulator = True
num_samples = 10
num_photons = 6

kernels = [None]*len(graphs)

for i in range(len(graphs)):
    # The mean photon number per mode m
    # is set to ensure that the singular values
    # are scaled such that all Sgates have squeezing value r=1
    num_singular_values = i//num_graphs+1
    A = graphs[i]
    m = 0.345274461385554870545 * num_singular_values
    
    prog = sf.Program(8)
    with prog.context as q:
        ops.BipartiteGraphEmbed(A, mean_photon_per_mode=m) | q
        ops.MeasureFock() | q

    #prog.compile("X8").print()
    
    if simulator:
        eng = sf.Engine("gaussian", backend_options={"cutoff_dim": 4})
        job = eng.run(prog, shots=num_samples)
        results=job # For simulator
    else:
        eng = sf.RemoteEngine("X8")
        job = eng.run(prog, shots=num_samples)

        while job.status != "complete":
            job.refresh()

        results=job.result #for hardware
    samples = results.samples
    #here we calculate the figerprint instead of storing all outcomes:
    kernels[i] = fingerprint(samples, num_photons)
#print(kernels)
print('Fingerprints with all orbits up to num_photons created')

/Users/loichenriet/Pasqal_code/xanadu/env-xanadu/lib/python3.7/site-packages/strawberryfields/backends/gaussianbackend/backend.py:232: UserWarning: Cannot simulate non-Gaussian states. Conditional state after Fock measurement has not been updated.


Fingerprints with all orbits up to num_photons created


In [5]:
### check that probabilites sum to 1
probs = []
for j in range(len(graphs)):
    full_proba = 0
    for i in kernels[j].keys():
        full_proba += kernels[j][i]
    probs.append(full_proba)
print('Max: '+str(np.max(probs)))
print('Min: '+str(np.min(probs)))

Max: 1.0000000000000002
Min: 0.9999999999999999


In [6]:
#Converting into vectors
vectorized_kernels = []
for kernel in kernels:
    vec_kernel = kernel_to_array(kernel)
    vectorized_kernels.append(vec_kernel)
vectorized_kernels = np.array(vectorized_kernels)

[0.625 0.    0.125 0.    0.    0.    0.    0.    0.    0.    0.    0.125
 0.    0.    0.    0.    0.125 0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.   ]
30


# Machine learning part. 

We generate a relatively large dataset, stored in the kernel_data file.

As we can provide each datapoint with a label (the number of non-zero singular values), we'll first try supervised learning.

In [4]:
vectorized_kernels = pickle.load( open( "kernel_data", "rb" ) )

kernels_training = []
kernels_testing = []
labels_training = []
labels_testing = []

for m in range(150):
    if m%3 == 0:
        kernels_testing.append(vectorized_kernels[m])
        labels_testing.append(m//50+1)
    else:
        kernels_training.append(vectorized_kernels[m])
        labels_training.append(m//50+1)

In [6]:
clf = svm.SVC()
clf.fit(kernels_training, labels_training)

SVC()

In [13]:
testing_data_accuracy = clf.predict(kernels_testing)
average = 0.
k = 0
tot = 0
for val in testing_data_accuracy:
    if np.isclose(val, labels_testing[k]):
        average += 1.
        
    k += 1
    tot += 1
average /= (50.)
print("Classification accuracy", average)

Classification accuracy 0.68


This gives us a 68% accuracy in the classification of the graphs. Let's now try an unsepervised learning approach.

In [14]:
num_graphs = 150
from sklearn.cluster import KMeans
X = vectorized_kernels
labels = [0]*num_graphs+[1]*num_graphs+[2]*num_graphs
kmeans = KMeans(n_clusters=3).fit(X)
fit = kmeans.fit(X)

In [16]:
labels_from_fit = fit.labels_
correct = 0
for i in range(len(labels_from_fit)):
    print
    if labels[i] == labels_from_fit[i]:
        correct +=1
correct = float(correct)/len(labels_from_fit)
correct

0.2733333333333333